In [19]:
import tensorflow as tf

import numpy as np
import os
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [20]:
#all your english are destroyed by us
#path_to_file = tf.keras.utils.get_file('All your english are destroyed by us.txt', 'https://raw.githubusercontent.com/JulieChandler-DS/ayeadbu/main/All%20your%20english%20are%20destroyed%20by%20us.txt')
#once and future king
path_to_file = tf.keras.utils.get_file('theonceandfutureking.txt', 'https://raw.githubusercontent.com/JulieChandler-DS/onceandfutureking/main/onceandfutureking.txt')

17965/17965 [==============================] - 0s 0us/step


In [21]:
  # Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 17946 characters


In [22]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿Incipit Liber Primus
THE SWORD IN THE STONE




She is not any common earth
  Water or wood or air,
But Merlin's Isle of Gramarye
  Where you and I will fare.






Chapter I
On Mondays, Wednesdays and Fridays it was Court Hand and 


In [23]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

67 unique characters


In [24]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [25]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [26]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[39, 40, 41, 42, 43, 44, 45], [62, 63, 64]]>

In [27]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [28]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [29]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [30]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [31]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(17946,), dtype=int64, numpy=array([67, 24, 52, ..., 50, 50,  9])>

In [32]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [33]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

﻿
I
n
c
i
p
i
t
 
L


In [34]:
seq_length = 100

In [35]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xef\xbb\xbf' b'I' b'n' b'c' b'i' b'p' b'i' b't' b' ' b'L' b'i' b'b'
 b'e' b'r' b' ' b'P' b'r' b'i' b'm' b'u' b's' b'\r' b'\n' b'T' b'H' b'E'
 b' ' b'S' b'W' b'O' b'R' b'D' b' ' b'I' b'N' b' ' b'T' b'H' b'E' b' '
 b'S' b'T' b'O' b'N' b'E' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'S' b'h' b'e' b' ' b'i' b's' b' ' b'n' b'o' b't' b' ' b'a'
 b'n' b'y' b' ' b'c' b'o' b'm' b'm' b'o' b'n' b' ' b'e' b'a' b'r' b't'
 b'h' b'\r' b'\n' b' ' b' ' b'W' b'a' b't' b'e' b'r' b' ' b'o' b'r' b' '
 b'w' b'o' b'o' b'd' b' ' b'o'], shape=(101,), dtype=string)


In [36]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\xef\xbb\xbfIncipit Liber Primus\r\nTHE SWORD IN THE STONE\r\n\r\n\r\n\r\n\r\nShe is not any common earth\r\n  Water or wood o'
b"r air,\r\nBut Merlin's Isle of Gramarye\r\n  Where you and I will fare.\r\n\r\n\r\n\r\n\r\n\r\n\r\nChapter I\r\nOn Monday"
b's, Wednesdays and Fridays it was Court Hand and Summulae Logicales, while the rest of the week it was'
b' the Organon, Repetition and Astrology. The governess was always getting muddled with her astrolabe, '
b'and when she got specially muddled she would take it out of the Wart by rapping his knuckles. She did'


In [37]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [38]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [39]:
dataset = sequences.map(split_input_target)

In [40]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xef\xbb\xbfIncipit Liber Primus\r\nTHE SWORD IN THE STONE\r\n\r\n\r\n\r\n\r\nShe is not any common earth\r\n  Water or wood '
Target: b'Incipit Liber Primus\r\nTHE SWORD IN THE STONE\r\n\r\n\r\n\r\n\r\nShe is not any common earth\r\n  Water or wood o'


In [41]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 17926

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [42]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [43]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.LSTM
    self.dense = tf.keras.layers.Dropout(.01)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [44]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [45]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 68) # (batch_size, sequence_length, vocab_size)


In [46]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  17408     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense_10 (Dense)            multiple                  69700     
                                                                 
Total params: 4,025,412
Trainable params: 4,025,412
Non-trainable params: 0
_________________________________________________________________


In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [48]:
sampled_indices

array([29, 37, 49,  8, 11,  1, 15, 31, 49, 60, 59,  9, 50, 18, 25, 25, 41,
       63, 13, 27, 13, 42, 19, 38, 30,  8, 60, 40, 11, 60, 18, 56, 16, 55,
        5, 42, 34, 61, 39,  7, 24, 53, 31, 55, 48, 21, 52, 65, 36, 47, 62,
        1, 61,  4, 34,  7, 20,  5, 10, 22,  5, 36, 11, 45, 31, 65, 45, 48,
       56, 27,  2, 43, 15, 29, 48,  4, 58, 40, 56, 53, 14, 32, 43,  3, 11,
        3,  1, 41,  3, 27, 13, 50,  4, 40, 26, 67,  2, 39, 49, 28])

In [49]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'drinking Metheglyn, not Port, but by mentioning the modern wine it is easier to give you the feel.\r\n'

Next Char Predictions:
 b'NWk-1\n?Pkvu.lCJJcy:L:dDYO-vb1vCrAq"dTwa,IoPqjFn\xc3\xa9Vix\nw!T,E"0G"V1gP\xc3\xa9gjrL\re?Nj!tbro;Re 1 \nc L:l!bK\xef\xbb\xbf\rakM'


In [50]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [51]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 68)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.2200804, shape=(), dtype=float32)


In [52]:
tf.exp(example_batch_mean_loss).numpy()

68.038956

In [53]:
model.compile(optimizer='adam', loss=loss)

In [54]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [55]:
EPOCHS = 120

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/120
2/2 [==============================] - 13s 5s/step - loss: 4.1989


In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Some woman next to id my vens out'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)